# 🚀 꿀스테이 RAG - 전체 파이프라인 통합 테스트

이 노트북에서는 1-5번 노트북의 모든 컴포넌트를 통합하여 완전한 RAG 시스템을 테스트합니다.

## 목표
1. 전체 RAG 파이프라인 통합
2. 엔드투엔드 성능 테스트
3. 실제 사용 시나리오 검증
4. 시스템 벤치마크 및 성능 분석
5. 최종 배포 준비 상태 확인

In [ ]:
import os
import sys
import time
import json
import uuid
from pathlib import Path
from typing import List, Dict, Any, Optional, Tuple, Union
import logging
from datetime import datetime, timedelta
from dataclasses import dataclass, field
from enum import Enum

# 환경 및 설정
from dotenv import load_dotenv

# LangChain 생태계
from langchain_openai import ChatOpenAI
from langchain_ollama import OllamaEmbeddings
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_core.documents import Document
from langchain.text_splitter import MarkdownHeaderTextSplitter
from langchain_community.tools import TavilySearchResults

# LangGraph
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver
from typing_extensions import TypedDict

# 데이터 분석 및 시각화
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets

# 환경변수 로드 (절대 경로 지정)
project_root = Path("/Users/yundoun/Desktop/Project/legal_rag/coolstay_rag")
env_file = project_root / ".env"
load_result = load_dotenv(env_file)
print(f"📝 .env 파일 로드: {load_result} (경로: {env_file})")

# API 키 확인
openai_key = os.getenv("OPENAI_API_KEY", "NOT_FOUND")
tavily_key = os.getenv("TAVILY_API_KEY", "NOT_FOUND")
print(f"🔑 OpenAI API Key: {'설정됨' if openai_key != 'NOT_FOUND' and openai_key.startswith('sk-') else 'NOT_FOUND'}")
print(f"🔑 Tavily API Key: {'설정됨' if tavily_key != 'NOT_FOUND' and tavily_key.startswith('tvly-') else 'NOT_FOUND'}")

# 로깅 설정
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# 시각화 설정
plt.rcParams['font.size'] = 10
plt.rcParams['figure.figsize'] = (12, 8)
sns.set_style("whitegrid")

print("✅ 라이브러리 import 완료")
print(f"📅 테스트 시작 시간: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

## 1. 통합 시스템 설정 및 초기화

In [ ]:
# 프로젝트 경로 및 설정
PROJECT_ROOT = Path("/Users/yundoun/Desktop/Project/legal_rag/coolstay_rag")
DATA_DIR = PROJECT_ROOT / "data"
CHROMA_DB_DIR = PROJECT_ROOT / "chroma_db"
NOTEBOOKS_DIR = PROJECT_ROOT / "notebooks"

# 도메인 설정 (전체 시스템 통합용)
DOMAIN_CONFIG = {
    "hr_policy": {
        "file": "HR_Policy_Guide.md",
        "description": "인사정책, 근무시간, 휴가, 급여, 복리후생",
        "collection_name": "hr_policy_db",
        "agent_name": "HR 정책 전문가",
        "keywords": ["인사", "HR", "근무", "휴가", "연차", "급여", "보험", "복리후생", "채용", "퇴사"]
    },
    "tech_policy": {
        "file": "Tech_Policy_Guide.md",
        "description": "기술정책, 개발환경, 코딩표준, 보안정책",
        "collection_name": "tech_policy_db",
        "agent_name": "기술 정책 전문가",
        "keywords": ["기술", "개발", "코딩", "코드", "프로그래밍", "보안", "개발환경", "IDE", "도구"]
    },
    "architecture": {
        "file": "Architecture_Guide.md",
        "description": "CMS 아키텍처, 시스템설계, 레이어구조",
        "collection_name": "architecture_db",
        "agent_name": "아키텍처 전문가",
        "keywords": ["아키텍처", "시스템", "설계", "구조", "레이어", "모듈", "서비스", "API", "데이터베이스"]
    },
    "component": {
        "file": "Component_Guide.md",
        "description": "컴포넌트 가이드라인, UI/UX 표준",
        "collection_name": "component_db",
        "agent_name": "컴포넌트 개발 전문가",
        "keywords": ["컴포넌트", "UI", "UX", "인터페이스", "디자인", "프론트엔드", "사용자", "화면"]
    },
    "deployment": {
        "file": "Deployment_Guide.md",
        "description": "배포프로세스, CI/CD, 환경관리",
        "collection_name": "deployment_db",
        "agent_name": "배포 전문가",
        "keywords": ["배포", "CI/CD", "환경", "서버", "클라우드", "도커", "쿠버네티스", "파이프라인"]
    },
    "development": {
        "file": "Development_Process_Guide.md",
        "description": "개발프로세스, 워크플로우, 협업규칙",
        "collection_name": "development_db",
        "agent_name": "개발 프로세스 전문가",
        "keywords": ["개발프로세스", "워크플로우", "협업", "프로세스", "방법론", "스크럼", "애자일"]
    },
    "business_policy": {
        "file": "Business_Policy_Guide.md",
        "description": "비즈니스정책, 운영규칙, 의사결정",
        "collection_name": "business_policy_db",
        "agent_name": "비즈니스 정책 전문가",
        "keywords": ["비즈니스", "정책", "운영", "의사결정", "전략", "계획", "목표", "성과"]
    },
    "web_search": {
        "description": "실시간 웹 검색을 통한 최신 정보 제공",
        "agent_name": "웹 검색 전문가",
        "keywords": ["최신", "트렌드", "뉴스", "동향", "현재", "실시간", "업데이트"]
    }
}

print(f"✅ 프로젝트 설정 완료")
print(f"   - 프로젝트 루트: {PROJECT_ROOT}")
print(f"   - 데이터 디렉토리: {DATA_DIR}")
print(f"   - ChromaDB 디렉토리: {CHROMA_DB_DIR}")
print(f"   - 도메인 수: {len(DOMAIN_CONFIG)}개")

In [ ]:
# 시스템 초기화 상태 추적
@dataclass
class SystemInitStatus:
    llm: bool = False
    embeddings: bool = False
    web_search: bool = False
    vectorstores: Dict[str, bool] = field(default_factory=dict)
    agents: Dict[str, bool] = field(default_factory=dict)
    
    def overall_status(self) -> float:
        """전체 초기화 성공률 계산"""
        total_components = 3 + len(DOMAIN_CONFIG) * 2  # LLM, Embeddings, WebSearch + (Vectorstore + Agent) per domain
        successful_components = (
            int(self.llm) + 
            int(self.embeddings) + 
            int(self.web_search) +
            sum(self.vectorstores.values()) +
            sum(self.agents.values())
        )
        return (successful_components / total_components) * 100

init_status = SystemInitStatus()

def initialize_system_components():
    """시스템 컴포넌트 통합 초기화"""
    
    print("🔧 시스템 컴포넌트 초기화 시작...\n")
    
    # 1. LLM 초기화
    print("1️⃣ LLM 초기화...")
    try:
        llm = ChatOpenAI(
            model="gpt-4o-mini",
            temperature=0.1,
            api_key=os.getenv("OPENAI_API_KEY")
        )
        # 테스트 호출
        test_response = llm.invoke("Hello")
        init_status.llm = True
        print(f"   ✅ LLM 초기화 성공: {llm.model_name}")
    except Exception as e:
        llm = None
        print(f"   ❌ LLM 초기화 실패: {e}")
    
    # 2. 임베딩 모델 초기화
    print("\n2️⃣ 임베딩 모델 초기화...")
    try:
        embeddings = OllamaEmbeddings(
            model="bge-m3",
            base_url="http://localhost:11434"
        )
        # 테스트 임베딩
        test_embedding = embeddings.embed_query("test")
        init_status.embeddings = True
        print(f"   ✅ 임베딩 모델 초기화 성공: bge-m3 ({len(test_embedding)}차원)")
    except Exception as e:
        embeddings = None
        print(f"   ❌ 임베딩 모델 초기화 실패: {e}")
    
    # 3. 웹 검색 도구 초기화
    print("\n3️⃣ 웹 검색 도구 초기화...")
    try:
        web_search = TavilySearchResults(
            max_results=3,
            api_key=os.getenv("TAVILY_API_KEY")
        )
        init_status.web_search = True
        print(f"   ✅ 웹 검색 도구 초기화 성공")
    except Exception as e:
        web_search = None
        print(f"   ❌ 웹 검색 도구 초기화 실패: {e}")
    
    # 4. 벡터 저장소 로딩
    print("\n4️⃣ 벡터 저장소 로딩...")
    vectorstores = {}
    
    if embeddings:
        for domain in DOMAIN_CONFIG.keys():
            if domain == "web_search":
                continue
                
            collection_name = DOMAIN_CONFIG[domain]["collection_name"]
            persist_directory = str(CHROMA_DB_DIR / domain)
            
            try:
                if Path(persist_directory).exists():
                    vectorstore = Chroma(
                        collection_name=collection_name,
                        embedding_function=embeddings,
                        persist_directory=persist_directory
                    )
                    
                    # 연결 테스트
                    count = vectorstore._collection.count()
                    vectorstores[domain] = vectorstore
                    init_status.vectorstores[domain] = True
                    print(f"   ✅ {domain}: {count}개 문서")
                else:
                    init_status.vectorstores[domain] = False
                    print(f"   ⚠️  {domain}: 벡터 저장소 없음")
                    
            except Exception as e:
                init_status.vectorstores[domain] = False
                print(f"   ❌ {domain}: 로딩 실패 - {e}")
    else:
        print("   ❌ 임베딩 모델이 없어 벡터 저장소를 로딩할 수 없습니다.")
    
    # 5. 에이전트 초기화 (간소화된 버전)
    print("\n5️⃣ RAG 에이전트 초기화...")
    agents = {}
    
    if llm:
        for domain in DOMAIN_CONFIG.keys():
            try:
                if domain == "web_search":
                    if web_search:
                        # 간소화된 웹 검색 에이전트
                        agents[domain] = {"type": "web_search", "tool": web_search, "llm": llm}
                        init_status.agents[domain] = True
                        print(f"   ✅ {domain}: 웹 검색 에이전트")
                    else:
                        init_status.agents[domain] = False
                        print(f"   ❌ {domain}: 웹 검색 도구 없음")
                else:
                    if domain in vectorstores:
                        # 간소화된 RAG 에이전트
                        agents[domain] = {
                            "type": "rag", 
                            "vectorstore": vectorstores[domain], 
                            "llm": llm,
                            "config": DOMAIN_CONFIG[domain]
                        }
                        init_status.agents[domain] = True
                        print(f"   ✅ {domain}: RAG 에이전트")
                    else:
                        init_status.agents[domain] = False
                        print(f"   ❌ {domain}: 벡터 저장소 없음")
                        
            except Exception as e:
                init_status.agents[domain] = False
                print(f"   ❌ {domain}: 에이전트 초기화 실패 - {e}")
    else:
        print("   ❌ LLM이 없어 에이전트를 초기화할 수 없습니다.")
    
    # 초기화 요약
    success_rate = init_status.overall_status()
    print(f"\n📊 시스템 초기화 완료")
    print(f"   - 전체 성공률: {success_rate:.1f}%")
    print(f"   - LLM: {'✅' if init_status.llm else '❌'}")
    print(f"   - 임베딩: {'✅' if init_status.embeddings else '❌'}")
    print(f"   - 웹 검색: {'✅' if init_status.web_search else '❌'}")
    print(f"   - 벡터 저장소: {sum(init_status.vectorstores.values())}/{len([d for d in DOMAIN_CONFIG if d != 'web_search'])}개")
    print(f"   - 에이전트: {sum(init_status.agents.values())}/{len(DOMAIN_CONFIG)}개")
    
    return {
        "llm": llm,
        "embeddings": embeddings,
        "web_search": web_search,
        "vectorstores": vectorstores,
        "agents": agents,
        "init_status": init_status
    }

# 시스템 초기화 실행
system_components = initialize_system_components()

## 2. 통합 RAG 파이프라인 클래스

In [ ]:
@dataclass
class PipelineTestResult:
    """파이프라인 테스트 결과"""
    test_id: str
    question: str
    answer: str
    selected_domains: List[str]
    processing_time: float
    success: bool
    error_message: Optional[str] = None
    
    # 상세 정보
    question_analysis_time: float = 0.0
    agent_execution_time: float = 0.0
    response_integration_time: float = 0.0
    
    # 품질 정보
    confidence_score: float = 0.0
    source_count: int = 0
    
    timestamp: datetime = field(default_factory=datetime.now)

class IntegratedRAGPipeline:
    """통합 RAG 파이프라인"""
    
    def __init__(self, system_components: Dict[str, Any]):
        self.llm = system_components["llm"]
        self.embeddings = system_components["embeddings"]
        self.web_search = system_components["web_search"]
        self.vectorstores = system_components["vectorstores"]
        self.agents = system_components["agents"]
        self.init_status = system_components["init_status"]
        
        # 질문 분석 프롬프트
        self.question_analysis_prompt = ChatPromptTemplate.from_template("""
        다음 질문을 분석하여 적절한 전문가 도메인을 선택해주세요.
        
        **질문:** {question}
        
        **사용 가능한 도메인:**
        - hr_policy: 인사정책, 근무시간, 휴가, 급여, 복리후생
        - tech_policy: 기술정책, 개발환경, 코딩표준, 보안정책
        - architecture: CMS 아키텍처, 시스템설계, 레이어구조
        - component: 컴포넌트 가이드라인, UI/UX 표준
        - deployment: 배포프로세스, CI/CD, 환경관리
        - development: 개발프로세스, 워크플로우, 협업규칙
        - business_policy: 비즈니스정책, 운영규칙, 의사결정
        - web_search: 실시간 웹 검색, 최신 정보
        
        최대 3개 도메인을 선택하고 JSON 형식으로 응답하세요:
        {{
            "selected_domains": ["domain1", "domain2"],
            "reasoning": "선택 근거"
        }}
        """)
        
        # 답변 통합 프롬프트
        self.integration_prompt = ChatPromptTemplate.from_template("""
        여러 전문가의 답변을 통합하여 최종 답변을 생성해주세요.
        
        **질문:** {question}
        
        **전문가 답변들:**
        {expert_answers}
        
        **통합 지침:**
        1. 각 전문가의 답변을 종합하여 완성도 높은 답변 생성
        2. 중복 내용 제거 및 일관성 유지
        3. 실용적이고 구체적인 정보 우선
        4. 한국어로 자연스럽게 작성
        
        통합된 최종 답변:
        """)
        
        # 체인 구성
        if self.llm:
            self.question_analysis_chain = (
                self.question_analysis_prompt | 
                self.llm | 
                JsonOutputParser()
            )
            
            self.integration_chain = (
                self.integration_prompt |
                self.llm |
                StrOutputParser()
            )
    
    def analyze_question(self, question: str) -> Tuple[List[str], str]:
        """질문 분석 및 도메인 선택"""
        if not self.llm:
            return ["hr_policy"], "LLM 없음 - 기본 도메인 사용"
        
        try:
            result = self.question_analysis_chain.invoke({"question": question})
            selected_domains = result.get("selected_domains", ["hr_policy"])
            reasoning = result.get("reasoning", "분석 완료")
            
            # 사용 가능한 도메인만 필터링
            available_domains = []
            for domain in selected_domains:
                if domain in self.agents:
                    available_domains.append(domain)
            
            return available_domains or ["hr_policy"], reasoning
            
        except Exception as e:
            logger.error(f"질문 분석 실패: {e}")
            return ["hr_policy"], f"분석 실패: {e}"
    
    def execute_agent(self, domain: str, question: str) -> Dict[str, Any]:
        """개별 에이전트 실행"""
        if domain not in self.agents:
            return {
                "success": False,
                "answer": f"{domain} 에이전트를 사용할 수 없습니다.",
                "source_count": 0,
                "confidence": 0.0
            }
        
        agent = self.agents[domain]
        
        try:
            if agent["type"] == "web_search":
                # 웹 검색 실행
                search_results = agent["tool"].invoke({"query": question})
                
                # 검색 결과 포맷팅 및 답변 생성
                if search_results:
                    formatted_results = "\n".join([
                        f"- {result.get('title', '')}: {result.get('content', '')[:100]}..."
                        for result in search_results[:3]
                    ])
                    
                    web_prompt = f"다음 웹 검색 결과를 바탕으로 '{question}'에 대한 답변을 한국어로 생성해주세요:\n\n{formatted_results}"
                    answer = agent["llm"].invoke(web_prompt).content
                    
                    return {
                        "success": True,
                        "answer": answer,
                        "source_count": len(search_results),
                        "confidence": 0.8
                    }
                else:
                    return {
                        "success": False,
                        "answer": "웹 검색 결과를 가져올 수 없습니다.",
                        "source_count": 0,
                        "confidence": 0.1
                    }
            
            else:  # RAG 에이전트
                vectorstore = agent["vectorstore"]
                config = agent["config"]
                
                # 문서 검색
                docs = vectorstore.similarity_search(question, k=3)
                
                if docs:
                    # 컨텍스트 구성
                    context = "\n\n".join([doc.page_content for doc in docs])
                    
                    # 답변 생성
                    agent_name = config.get("agent_name", f"{domain} 전문가")
                    description = config.get("description", "전문가")
                    
                    rag_prompt = f"""
                    당신은 꿀스테이의 {agent_name}입니다.
                    전문 분야: {description}
                    
                    다음 문서 정보를 바탕으로 질문에 답변해주세요.
                    
                    **질문:** {question}
                    
                    **관련 문서:**
                    {context}
                    
                    **답변:**
                    """
                    
                    answer = agent["llm"].invoke(rag_prompt).content
                    
                    return {
                        "success": True,
                        "answer": answer,
                        "source_count": len(docs),
                        "confidence": 0.9
                    }
                else:
                    return {
                        "success": False,
                        "answer": f"{domain} 도메인에서 관련 정보를 찾을 수 없습니다.",
                        "source_count": 0,
                        "confidence": 0.2
                    }
        
        except Exception as e:
            logger.error(f"에이전트 실행 실패 ({domain}): {e}")
            return {
                "success": False,
                "answer": f"{domain} 에이전트 실행 중 오류 발생: {e}",
                "source_count": 0,
                "confidence": 0.0
            }
    
    def integrate_responses(self, question: str, domain_responses: Dict[str, Dict]) -> str:
        """다중 에이전트 응답 통합"""
        if not self.llm:
            # LLM이 없는 경우 첫 번째 성공한 응답 반환
            for domain, response in domain_responses.items():
                if response["success"]:
                    return response["answer"]
            return "답변을 생성할 수 없습니다."
        
        if len(domain_responses) == 1:
            # 단일 응답인 경우 통합 없이 직접 반환
            return list(domain_responses.values())[0]["answer"]
        
        try:
            # 성공한 응답들만 수집
            successful_responses = {
                domain: response for domain, response in domain_responses.items()
                if response["success"]
            }
            
            if not successful_responses:
                return "죄송합니다. 답변을 생성할 수 없습니다."
            
            # 전문가 답변 포맷팅
            expert_answers = []
            for domain, response in successful_responses.items():
                agent_name = DOMAIN_CONFIG.get(domain, {}).get("agent_name", f"{domain} 전문가")
                expert_answers.append(f"**{agent_name}:** {response['answer']}")
            
            expert_answers_text = "\n\n".join(expert_answers)
            
            # 통합 답변 생성
            integrated_answer = self.integration_chain.invoke({
                "question": question,
                "expert_answers": expert_answers_text
            })
            
            return integrated_answer
            
        except Exception as e:
            logger.error(f"응답 통합 실패: {e}")
            # 실패 시 첫 번째 성공한 응답 반환
            for response in domain_responses.values():
                if response["success"]:
                    return response["answer"]
            return f"응답 통합 중 오류 발생: {e}"
    
    def process_question(self, question: str) -> PipelineTestResult:
        """전체 파이프라인 실행"""
        test_id = str(uuid.uuid4())[:8]
        start_time = time.time()
        
        try:
            # 1. 질문 분석
            analysis_start = time.time()
            selected_domains, reasoning = self.analyze_question(question)
            analysis_time = time.time() - analysis_start
            
            # 2. 에이전트 실행
            execution_start = time.time()
            domain_responses = {}
            
            for domain in selected_domains:
                response = self.execute_agent(domain, question)
                domain_responses[domain] = response
            
            execution_time = time.time() - execution_start
            
            # 3. 응답 통합
            integration_start = time.time()
            final_answer = self.integrate_responses(question, domain_responses)
            integration_time = time.time() - integration_start
            
            # 통계 계산
            total_time = time.time() - start_time
            successful_responses = [r for r in domain_responses.values() if r["success"]]
            avg_confidence = sum(r["confidence"] for r in successful_responses) / len(successful_responses) if successful_responses else 0.0
            total_sources = sum(r["source_count"] for r in domain_responses.values())
            
            return PipelineTestResult(
                test_id=test_id,
                question=question,
                answer=final_answer,
                selected_domains=selected_domains,
                processing_time=total_time,
                success=True,
                question_analysis_time=analysis_time,
                agent_execution_time=execution_time,
                response_integration_time=integration_time,
                confidence_score=avg_confidence,
                source_count=total_sources
            )
            
        except Exception as e:
            logger.error(f"파이프라인 실행 실패: {e}")
            return PipelineTestResult(
                test_id=test_id,
                question=question,
                answer=f"처리 중 오류 발생: {e}",
                selected_domains=[],
                processing_time=time.time() - start_time,
                success=False,
                error_message=str(e)
            )

# 통합 파이프라인 초기화
if system_components["init_status"].overall_status() > 50:
    integrated_pipeline = IntegratedRAGPipeline(system_components)
    print("✅ 통합 RAG 파이프라인 초기화 완료")
else:
    integrated_pipeline = None
    print(f"❌ 시스템 초기화 상태가 불충분합니다 ({system_components['init_status'].overall_status():.1f}%)")
    print("   통합 파이프라인을 실행하려면 더 많은 컴포넌트가 필요합니다.")

## 3. 종합 성능 테스트 시나리오

In [ ]:
# 종합 테스트 시나리오
comprehensive_test_scenarios = [
    {
        "category": "단일 도메인",
        "questions": [
            "연차 휴가는 어떻게 신청하나요?",
            "코딩 스타일 가이드라인을 알려주세요",
            "시스템 아키텍처의 주요 구성요소는 무엇인가요?",
            "배포 프로세스는 어떻게 진행하나요?"
        ]
    },
    {
        "category": "다중 도메인",
        "questions": [
            "개발자 채용부터 온보딩까지의 전체 프로세스를 설명해주세요",
            "새로운 기능 개발 시 아키텍처 설계부터 배포까지 어떤 절차를 거치나요?",
            "회사의 기술 스택과 개발 문화에 대해 알려주세요",
            "UI 컴포넌트 개발 및 배포 가이드라인은 무엇인가요?"
        ]
    },
    {
        "category": "일반적 질문",
        "questions": [
            "회사의 핵심 가치와 문화는 무엇인가요?",
            "새로 입사한 개발자가 알아야 할 것들은 무엇인가요?",
            "프로젝트 관리는 어떻게 이루어지나요?",
            "회사의 성장 전략과 비전을 설명해주세요"
        ]
    },
    {
        "category": "최신 정보",
        "questions": [
            "2024년 최신 웹 개발 트렌드는 무엇인가요?",
            "AI 기술의 최근 동향을 알려주세요",
            "클라우드 컴퓨팅의 최신 발전사항은 무엇인가요?",
            "현재 인기있는 프로그래밍 언어는 무엇인가요?"
        ]
    }
]

def run_comprehensive_test(pipeline: IntegratedRAGPipeline, 
                         scenarios: List[Dict],
                         max_questions_per_category: int = 2) -> List[PipelineTestResult]:
    """종합 성능 테스트 실행"""
    
    print("🚀 종합 성능 테스트 시작")
    print(f"   - 테스트 카테고리: {len(scenarios)}개")
    print(f"   - 카테고리당 최대 질문: {max_questions_per_category}개\n")
    
    all_results = []
    
    for scenario in scenarios:
        category = scenario["category"]
        questions = scenario["questions"][:max_questions_per_category]
        
        print(f"📋 카테고리: {category}")
        print("="*50)
        
        category_results = []
        
        for i, question in enumerate(questions, 1):
            print(f"\n{i}️⃣ 질문: {question}")
            
            try:
                # 파이프라인 실행
                result = pipeline.process_question(question)
                
                # 결과 출력
                print(f"   ✅ 성공: {result.success}")
                print(f"   🎯 선택 도메인: {', '.join(result.selected_domains)}")
                print(f"   ⏱️  처리 시간: {result.processing_time:.2f}초")
                print(f"   📊 신뢰도: {result.confidence_score:.2f}")
                print(f"   📚 출처 수: {result.source_count}개")
                
                # 답변 미리보기
                answer_preview = result.answer[:100] + "..." if len(result.answer) > 100 else result.answer
                print(f"   💬 답변: {answer_preview}")
                
                category_results.append(result)
                all_results.append(result)
                
            except Exception as e:
                print(f"   ❌ 실패: {e}")
                logger.error(f"테스트 실패 ({category} - {question}): {e}")
        
        # 카테고리 요약
        if category_results:
            success_count = sum(1 for r in category_results if r.success)
            avg_time = sum(r.processing_time for r in category_results) / len(category_results)
            avg_confidence = sum(r.confidence_score for r in category_results) / len(category_results)
            
            print(f"\n📊 {category} 요약:")
            print(f"   - 성공률: {success_count}/{len(category_results)} ({success_count/len(category_results)*100:.1f}%)")
            print(f"   - 평균 처리 시간: {avg_time:.2f}초")
            print(f"   - 평균 신뢰도: {avg_confidence:.2f}")
        
        print("\n" + "="*70 + "\n")
    
    return all_results

# 종합 테스트 실행
if integrated_pipeline:
    print(f"📊 시스템 준비 상태: {system_components['init_status'].overall_status():.1f}%\n")
    
    comprehensive_results = run_comprehensive_test(
        pipeline=integrated_pipeline,
        scenarios=comprehensive_test_scenarios,
        max_questions_per_category=2  # 성능을 위해 각 카테고리당 2개 질문만 테스트
    )
    
    print(f"🎉 종합 성능 테스트 완료: {len(comprehensive_results)}개 테스트")
    
else:
    comprehensive_results = []
    print("❌ 통합 파이프라인이 준비되지 않아 종합 테스트를 건너뜁니다.")

## 4. 성능 분석 및 벤치마크

In [ ]:
class PerformanceAnalyzer:
    """성능 분석 및 벤치마크"""
    
    def __init__(self, test_results: List[PipelineTestResult]):
        self.test_results = test_results
        self.df = self._create_dataframe()
    
    def _create_dataframe(self) -> pd.DataFrame:
        """테스트 결과를 DataFrame으로 변환"""
        if not self.test_results:
            return pd.DataFrame()
        
        data = []
        for result in self.test_results:
            row = {
                'test_id': result.test_id,
                'question': result.question[:50] + '...' if len(result.question) > 50 else result.question,
                'success': result.success,
                'domains_count': len(result.selected_domains),
                'selected_domains': ', '.join(result.selected_domains),
                'processing_time': result.processing_time,
                'analysis_time': result.question_analysis_time,
                'execution_time': result.agent_execution_time,
                'integration_time': result.response_integration_time,
                'confidence_score': result.confidence_score,
                'source_count': result.source_count,
                'answer_length': len(result.answer),
                'timestamp': result.timestamp
            }
            data.append(row)
        
        return pd.DataFrame(data)
    
    def generate_performance_report(self) -> str:
        """성능 보고서 생성"""
        if self.df.empty:
            return "분석할 데이터가 없습니다."
        
        successful_tests = self.df[self.df['success'] == True]
        
        report = []
        report.append("🚀 꿀스테이 RAG 시스템 성능 분석 보고서")
        report.append("=" * 60)
        
        # 기본 통계
        report.append(f"\n📊 기본 통계:")
        report.append(f"   - 총 테스트 수: {len(self.df)}개")
        report.append(f"   - 성공한 테스트: {len(successful_tests)}개")
        report.append(f"   - 성공률: {len(successful_tests)/len(self.df)*100:.1f}%")
        
        if len(successful_tests) > 0:
            # 성능 지표
            report.append(f"\n⚡ 성능 지표 (성공한 테스트만):")
            report.append(f"   - 평균 처리 시간: {successful_tests['processing_time'].mean():.2f}초")
            report.append(f"   - 최대 처리 시간: {successful_tests['processing_time'].max():.2f}초")
            report.append(f"   - 최소 처리 시간: {successful_tests['processing_time'].min():.2f}초")
            report.append(f"   - 처리 시간 표준편차: {successful_tests['processing_time'].std():.2f}초")
            
            # 처리 단계별 시간 분석
            report.append(f"\n🔍 처리 단계별 평균 시간:")
            if 'analysis_time' in successful_tests.columns:
                report.append(f"   - 질문 분석: {successful_tests['analysis_time'].mean():.3f}초")
            if 'execution_time' in successful_tests.columns:
                report.append(f"   - 에이전트 실행: {successful_tests['execution_time'].mean():.3f}초")
            if 'integration_time' in successful_tests.columns:
                report.append(f"   - 응답 통합: {successful_tests['integration_time'].mean():.3f}초")
            
            # 품질 지표
            report.append(f"\n🎯 품질 지표:")
            report.append(f"   - 평균 신뢰도: {successful_tests['confidence_score'].mean():.2f}")
            report.append(f"   - 평균 출처 수: {successful_tests['source_count'].mean():.1f}개")
            report.append(f"   - 평균 답변 길이: {successful_tests['answer_length'].mean():.0f}자")
            
            # 도메인 사용 분석
            report.append(f"\n🏷️  도메인 사용 분석:")
            domain_usage = successful_tests['domains_count'].value_counts().sort_index()
            for domain_count, usage_count in domain_usage.items():
                percentage = (usage_count / len(successful_tests)) * 100
                report.append(f"   - {domain_count}개 도메인 사용: {usage_count}회 ({percentage:.1f}%)")
            
            # 자주 사용된 도메인
            all_domains = []
            for domains_str in successful_tests['selected_domains']:
                all_domains.extend(domains_str.split(', '))
            
            domain_frequency = pd.Series(all_domains).value_counts()
            report.append(f"\n📈 도메인 사용 빈도:")
            for domain, count in domain_frequency.head(5).items():
                percentage = (count / len(successful_tests)) * 100
                report.append(f"   - {domain}: {count}회 ({percentage:.1f}%)")
        
        # 실패 분석
        failed_tests = self.df[self.df['success'] == False]
        if len(failed_tests) > 0:
            report.append(f"\n❌ 실패 분석:")
            report.append(f"   - 실패한 테스트: {len(failed_tests)}개")
            report.append(f"   - 실패율: {len(failed_tests)/len(self.df)*100:.1f}%")
        
        return "\n".join(report)
    
    def plot_performance_metrics(self, figsize=(15, 10)):
        """성능 지표 시각화"""
        if self.df.empty:
            print("시각화할 데이터가 없습니다.")
            return
        
        successful_tests = self.df[self.df['success'] == True]
        
        if len(successful_tests) == 0:
            print("성공한 테스트가 없어 시각화할 수 없습니다.")
            return
        
        fig, axes = plt.subplots(2, 3, figsize=figsize)
        fig.suptitle('꿀스테이 RAG 시스템 성능 분석', fontsize=16)
        
        # 1. 처리 시간 분포
        axes[0, 0].hist(successful_tests['processing_time'], bins=10, alpha=0.7, color='skyblue', edgecolor='black')
        axes[0, 0].set_title('Processing Time Distribution')
        axes[0, 0].set_xlabel('Processing Time (seconds)')
        axes[0, 0].set_ylabel('Frequency')
        
        # 2. 신뢰도 분포
        axes[0, 1].hist(successful_tests['confidence_score'], bins=10, alpha=0.7, color='lightgreen', edgecolor='black')
        axes[0, 1].set_title('Confidence Score Distribution')
        axes[0, 1].set_xlabel('Confidence Score')
        axes[0, 1].set_ylabel('Frequency')
        
        # 3. 도메인 사용 수 분포
        domain_counts = successful_tests['domains_count'].value_counts().sort_index()
        axes[0, 2].bar(domain_counts.index, domain_counts.values, color='orange', alpha=0.7)
        axes[0, 2].set_title('Number of Domains Used')
        axes[0, 2].set_xlabel('Number of Domains')
        axes[0, 2].set_ylabel('Frequency')
        
        # 4. 처리 시간 vs 신뢰도
        axes[1, 0].scatter(successful_tests['processing_time'], successful_tests['confidence_score'], 
                          alpha=0.6, color='purple')
        axes[1, 0].set_title('Processing Time vs Confidence')
        axes[1, 0].set_xlabel('Processing Time (seconds)')
        axes[1, 0].set_ylabel('Confidence Score')
        
        # 5. 출처 수 분포
        axes[1, 1].hist(successful_tests['source_count'], bins=10, alpha=0.7, color='pink', edgecolor='black')
        axes[1, 1].set_title('Source Count Distribution')
        axes[1, 1].set_xlabel('Number of Sources')
        axes[1, 1].set_ylabel('Frequency')
        
        # 6. 성공/실패 파이 차트
        success_counts = self.df['success'].value_counts()
        axes[1, 2].pie(success_counts.values, labels=['Success', 'Failure'], autopct='%1.1f%%',
                      colors=['lightgreen', 'lightcoral'])
        axes[1, 2].set_title('Success Rate')
        
        plt.tight_layout()
        plt.show()
    
    def create_benchmark_summary(self) -> Dict[str, Any]:
        """벤치마크 요약 생성"""
        if self.df.empty:
            return {"error": "데이터 없음"}
        
        successful_tests = self.df[self.df['success'] == True]
        
        return {
            "system_info": {
                "total_tests": len(self.df),
                "successful_tests": len(successful_tests),
                "success_rate": len(successful_tests) / len(self.df) * 100 if len(self.df) > 0 else 0,
                "test_date": datetime.now().isoformat()
            },
            "performance_metrics": {
                "avg_processing_time": successful_tests['processing_time'].mean() if len(successful_tests) > 0 else 0,
                "max_processing_time": successful_tests['processing_time'].max() if len(successful_tests) > 0 else 0,
                "min_processing_time": successful_tests['processing_time'].min() if len(successful_tests) > 0 else 0,
                "std_processing_time": successful_tests['processing_time'].std() if len(successful_tests) > 0 else 0
            },
            "quality_metrics": {
                "avg_confidence_score": successful_tests['confidence_score'].mean() if len(successful_tests) > 0 else 0,
                "avg_source_count": successful_tests['source_count'].mean() if len(successful_tests) > 0 else 0,
                "avg_answer_length": successful_tests['answer_length'].mean() if len(successful_tests) > 0 else 0
            },
            "component_status": {
                "llm_available": system_components["init_status"].llm,
                "embeddings_available": system_components["init_status"].embeddings,
                "web_search_available": system_components["init_status"].web_search,
                "vectorstores_count": sum(system_components["init_status"].vectorstores.values()),
                "agents_count": sum(system_components["init_status"].agents.values())
            }
        }

# 성능 분석 실행
if comprehensive_results:
    print("📊 성능 분석 및 벤치마크 생성 중...\n")
    
    analyzer = PerformanceAnalyzer(comprehensive_results)
    
    # 성능 보고서 출력
    performance_report = analyzer.generate_performance_report()
    print(performance_report)
    
    print("\n" + "="*70)
    print("📈 성능 지표 시각화:")
    
    # 성능 차트 생성
    analyzer.plot_performance_metrics(figsize=(16, 10))
    
    # 벤치마크 요약
    benchmark_summary = analyzer.create_benchmark_summary()
    
    print("\n📋 벤치마크 요약 (JSON):")
    print(json.dumps(benchmark_summary, indent=2, ensure_ascii=False))
    
else:
    print("❌ 분석할 테스트 결과가 없습니다.")

## 5. 상세 테스트 결과 표시

In [ ]:
# 상세 테스트 결과 표시
if comprehensive_results:
    print("📋 상세 테스트 결과\n")
    
    # DataFrame으로 변환하여 표시
    results_df = analyzer.df
    
    # 주요 컬럼만 선택해서 표시
    display_columns = [
        'test_id', 'question', 'success', 'selected_domains',
        'processing_time', 'confidence_score', 'source_count'
    ]
    
    display_df = results_df[display_columns].copy()
    
    # 시간 형식 정리
    display_df['processing_time'] = display_df['processing_time'].round(2)
    display_df['confidence_score'] = display_df['confidence_score'].round(2)
    
    print("상위 결과:")
    display(display_df.head(10))
    
    # 성공한 테스트의 답변 미리보기
    print("\n💬 성공한 테스트의 답변 미리보기:")
    print("="*60)
    
    successful_results = [r for r in comprehensive_results if r.success]
    
    for i, result in enumerate(successful_results[:3], 1):  # 처음 3개만 표시
        print(f"\n{i}. 질문: {result.question}")
        print(f"   도메인: {', '.join(result.selected_domains)}")
        print(f"   처리시간: {result.processing_time:.2f}초")
        print(f"   답변 미리보기:")
        
        # 답변을 200자로 제한하여 표시
        answer_preview = result.answer[:200] + "..." if len(result.answer) > 200 else result.answer
        print(f"   {answer_preview}")
        print("-" * 60)
    
    # 실패한 테스트 분석
    failed_results = [r for r in comprehensive_results if not r.success]
    if failed_results:
        print(f"\n❌ 실패한 테스트 분석 ({len(failed_results)}개):")
        print("="*60)
        
        for i, result in enumerate(failed_results[:3], 1):  # 처음 3개만 표시
            print(f"\n{i}. 질문: {result.question}")
            print(f"   오류: {result.error_message or '알 수 없는 오류'}")
            print(f"   처리시간: {result.processing_time:.2f}초")
            print("-" * 60)
    
else:
    print("❌ 표시할 테스트 결과가 없습니다.")

## 6. 시스템 상태 및 배포 준비도 점검

In [ ]:
class DeploymentReadinessChecker:
    """배포 준비도 점검기"""
    
    def __init__(self, init_status: SystemInitStatus, test_results: List[PipelineTestResult]):
        self.init_status = init_status
        self.test_results = test_results
    
    def check_component_readiness(self) -> Dict[str, Dict[str, Any]]:
        """컴포넌트별 준비 상태 점검"""
        
        checks = {
            "core_components": {
                "llm_available": {
                    "status": self.init_status.llm,
                    "importance": "Critical",
                    "description": "OpenAI GPT-4o-mini 언어 모델"
                },
                "embeddings_available": {
                    "status": self.init_status.embeddings,
                    "importance": "Critical",
                    "description": "Ollama bge-m3 임베딩 모델"
                },
                "web_search_available": {
                    "status": self.init_status.web_search,
                    "importance": "Optional",
                    "description": "Tavily API 웹 검색 도구"
                }
            },
            "data_components": {},
            "agent_components": {}
        }
        
        # 벡터 저장소 점검
        for domain, status in self.init_status.vectorstores.items():
            checks["data_components"][f"{domain}_vectorstore"] = {
                "status": status,
                "importance": "High" if domain in ["hr_policy", "tech_policy"] else "Medium",
                "description": f"{domain} 도메인 벡터 저장소"
            }
        
        # 에이전트 점검
        for domain, status in self.init_status.agents.items():
            checks["agent_components"][f"{domain}_agent"] = {
                "status": status,
                "importance": "High" if domain in ["hr_policy", "tech_policy"] else "Medium",
                "description": f"{domain} 도메인 RAG 에이전트"
            }
        
        return checks
    
    def check_performance_readiness(self) -> Dict[str, Any]:
        """성능 준비 상태 점검"""
        if not self.test_results:
            return {
                "test_coverage": False,
                "success_rate": 0.0,
                "avg_processing_time": 0.0,
                "performance_acceptable": False
            }
        
        successful_tests = [r for r in self.test_results if r.success]
        success_rate = len(successful_tests) / len(self.test_results) * 100
        
        avg_processing_time = 0.0
        if successful_tests:
            avg_processing_time = sum(r.processing_time for r in successful_tests) / len(successful_tests)
        
        return {
            "test_coverage": len(self.test_results) >= 5,  # 최소 5개 테스트
            "success_rate": success_rate,
            "avg_processing_time": avg_processing_time,
            "performance_acceptable": success_rate >= 80 and avg_processing_time <= 10.0  # 80% 성공률, 10초 이내
        }
    
    def generate_readiness_report(self) -> str:
        """배포 준비도 보고서 생성"""
        component_checks = self.check_component_readiness()
        performance_checks = self.check_performance_readiness()
        
        report = []
        report.append("🚀 꿀스테이 RAG 시스템 배포 준비도 점검")
        report.append("=" * 60)
        
        # 전체 점수 계산
        total_components = 0
        ready_components = 0
        critical_ready = 0
        critical_total = 0
        
        # 컴포넌트별 점검 결과
        for category, components in component_checks.items():
            report.append(f"\n📊 {category.replace('_', ' ').title()}:")
            
            for comp_name, comp_info in components.items():
                status_icon = "✅" if comp_info["status"] else "❌"
                importance = comp_info["importance"]
                description = comp_info["description"]
                
                report.append(f"   {status_icon} {comp_name}: {description} ({importance})")
                
                total_components += 1
                if comp_info["status"]:
                    ready_components += 1
                
                if importance == "Critical":
                    critical_total += 1
                    if comp_info["status"]:
                        critical_ready += 1
        
        # 성능 점검 결과
        report.append(f"\n🎯 성능 점검:")
        report.append(f"   {'✅' if performance_checks['test_coverage'] else '❌'} 테스트 커버리지: {len(self.test_results)}개 테스트")
        report.append(f"   {'✅' if performance_checks['success_rate'] >= 80 else '❌'} 성공률: {performance_checks['success_rate']:.1f}% (목표: ≥80%)")
        report.append(f"   {'✅' if performance_checks['avg_processing_time'] <= 10.0 else '❌'} 평균 처리시간: {performance_checks['avg_processing_time']:.2f}초 (목표: ≤10초)")
        
        # 전체 준비도 계산
        component_readiness = (ready_components / total_components * 100) if total_components > 0 else 0
        critical_readiness = (critical_ready / critical_total * 100) if critical_total > 0 else 0
        
        report.append(f"\n📈 전체 준비도:")
        report.append(f"   - 전체 컴포넌트: {ready_components}/{total_components} ({component_readiness:.1f}%)")
        report.append(f"   - 핵심 컴포넌트: {critical_ready}/{critical_total} ({critical_readiness:.1f}%)")
        report.append(f"   - 성능 준비도: {'✅' if performance_checks['performance_acceptable'] else '❌'}")
        
        # 배포 권장사항
        report.append(f"\n🎯 배포 권장사항:")
        
        if critical_readiness == 100 and performance_checks['performance_acceptable']:
            report.append("   🟢 배포 준비 완료! 프로덕션 환경에 배포할 수 있습니다.")
        elif critical_readiness == 100:
            report.append("   🟡 핵심 기능은 준비되었으나 성능 최적화가 필요합니다.")
            report.append("   📝 권장: 성능 튜닝 후 배포")
        elif critical_readiness >= 50:
            report.append("   🟠 일부 핵심 컴포넌트가 준비되지 않았습니다.")
            report.append("   📝 권장: 스테이징 환경에서 추가 테스트")
        else:
            report.append("   🔴 배포 준비가 부족합니다.")
            report.append("   📝 권장: 핵심 컴포넌트 수정 후 재테스트")
        
        # 개선 항목
        improvements = []
        
        for category, components in component_checks.items():
            for comp_name, comp_info in components.items():
                if not comp_info["status"] and comp_info["importance"] in ["Critical", "High"]:
                    improvements.append(f"   - {comp_info['description']} 활성화 필요")
        
        if not performance_checks['performance_acceptable']:
            if performance_checks['success_rate'] < 80:
                improvements.append("   - 성공률 향상 필요 (현재: {:.1f}%, 목표: ≥80%)".format(performance_checks['success_rate']))
            if performance_checks['avg_processing_time'] > 10.0:
                improvements.append("   - 처리 시간 단축 필요 (현재: {:.2f}초, 목표: ≤10초)".format(performance_checks['avg_processing_time']))
        
        if improvements:
            report.append(f"\n🔧 개선 필요 항목:")
            report.extend(improvements)
        
        return "\n".join(report)
    
    def get_deployment_score(self) -> float:
        """배포 준비도 점수 (0-100)"""
        component_checks = self.check_component_readiness()
        performance_checks = self.check_performance_readiness()
        
        # 컴포넌트 점수 계산 (가중치 적용)
        component_score = 0.0
        total_weight = 0.0
        
        for category, components in component_checks.items():
            for comp_name, comp_info in components.items():
                weight = 3.0 if comp_info["importance"] == "Critical" else (
                    2.0 if comp_info["importance"] == "High" else 1.0
                )
                
                if comp_info["status"]:
                    component_score += weight
                total_weight += weight
        
        component_ratio = component_score / total_weight if total_weight > 0 else 0.0
        
        # 성능 점수 계산
        performance_score = 0.0
        if performance_checks['test_coverage']:
            performance_score += 0.2
        if performance_checks['success_rate'] >= 80:
            performance_score += 0.4
        if performance_checks['avg_processing_time'] <= 10.0:
            performance_score += 0.4
        
        # 전체 점수 (컴포넌트 70%, 성능 30%)
        total_score = (component_ratio * 0.7 + performance_score * 0.3) * 100
        
        return min(100.0, max(0.0, total_score))

# 배포 준비도 점검
if system_components["init_status"] and comprehensive_results:
    print("🔍 배포 준비도 점검 중...\n")
    
    readiness_checker = DeploymentReadinessChecker(
        init_status=system_components["init_status"],
        test_results=comprehensive_results
    )
    
    # 배포 준비도 보고서
    readiness_report = readiness_checker.generate_readiness_report()
    print(readiness_report)
    
    # 배포 점수
    deployment_score = readiness_checker.get_deployment_score()
    
    print(f"\n⭐ 최종 배포 준비도 점수: {deployment_score:.1f}/100점")
    
    # 점수별 권장사항
    if deployment_score >= 90:
        print("🟢 우수! 프로덕션 배포 준비 완료")
    elif deployment_score >= 70:
        print("🟡 양호! 일부 개선 후 배포 가능")
    elif deployment_score >= 50:
        print("🟠 보통! 주요 개선사항 해결 필요")
    else:
        print("🔴 미흡! 전면적인 시스템 점검 필요")
    
else:
    print("❌ 배포 준비도를 점검할 데이터가 충분하지 않습니다.")

## 7. 최종 요약 및 권장사항

In [ ]:
def generate_final_summary() -> str:
    """최종 요약 보고서 생성"""
    
    current_time = datetime.now()
    
    summary = []
    summary.append("🍯 꿀스테이 RAG 시스템 - 최종 통합 테스트 보고서")
    summary.append("=" * 80)
    summary.append(f"📅 테스트 완료 시간: {current_time.strftime('%Y-%m-%d %H:%M:%S')}")
    
    # 시스템 개요
    summary.append(f"\n🏗️  시스템 개요:")
    summary.append(f"   - 아키텍처: Multi-Agent RAG with Human-in-the-Loop")
    summary.append(f"   - 도메인: 7개 전문 도메인 + 웹 검색")
    summary.append(f"   - 핵심 기술: LangChain, LangGraph, ChromaDB, OpenAI GPT-4o-mini")
    summary.append(f"   - 임베딩: Ollama bge-m3 (다국어 지원)")
    
    # 구현 완료 현황
    summary.append(f"\n✅ 구현 완료 현황:")
    summary.append(f"   📝 01_data_processing: 문서 로딩 및 청킹 시스템")
    summary.append(f"   📊 02_vector_stores: 도메인별 독립 벡터 저장소")
    summary.append(f"   🤖 03_agents_development: Corrective RAG 에이전트 (8개)")
    summary.append(f"   🧭 04_routing_integration: 질문 라우팅 및 답변 통합")
    summary.append(f"   🧑‍⚖️ 05_hitl_evaluation: ReAct 평가 및 HITL 시스템")
    summary.append(f"   🚀 99_full_pipeline_test: 전체 통합 테스트")
    
    # 핵심 기능
    summary.append(f"\n🎯 핵심 기능:")
    summary.append(f"   🔍 지능형 질문 분석 및 도메인 라우팅")
    summary.append(f"   📚 도메인별 전문 지식 검색 (7개 도메인)")
    summary.append(f"   🔄 Corrective RAG (자가교정 메커니즘)")
    summary.append(f"   🌐 실시간 웹 검색 통합 (Tavily API)")
    summary.append(f"   🧠 다중 에이전트 답변 통합")
    summary.append(f"   ⚖️ 6차원 품질 평가 시스템 (60점 만점)")
    summary.append(f"   👥 Human-in-the-Loop 품질 검증")
    summary.append(f"   📊 실시간 성능 모니터링")
    
    # 시스템 현황
    if system_components["init_status"]:
        init_status = system_components["init_status"]
        overall_status = init_status.overall_status()
        
        summary.append(f"\n📊 시스템 현황:")
        summary.append(f"   - 전체 초기화 성공률: {overall_status:.1f}%")
        summary.append(f"   - LLM (GPT-4o-mini): {'✅' if init_status.llm else '❌'}")
        summary.append(f"   - 임베딩 (bge-m3): {'✅' if init_status.embeddings else '❌'}")
        summary.append(f"   - 웹 검색 (Tavily): {'✅' if init_status.web_search else '❌'}")
        summary.append(f"   - 벡터 저장소: {sum(init_status.vectorstores.values())}/7개")
        summary.append(f"   - RAG 에이전트: {sum(init_status.agents.values())}/8개")
    
    # 성능 결과
    if comprehensive_results:
        successful_tests = [r for r in comprehensive_results if r.success]
        success_rate = len(successful_tests) / len(comprehensive_results) * 100
        
        if successful_tests:
            avg_time = sum(r.processing_time for r in successful_tests) / len(successful_tests)
            avg_confidence = sum(r.confidence_score for r in successful_tests) / len(successful_tests)
            
            summary.append(f"\n🎯 성능 결과:")
            summary.append(f"   - 총 테스트: {len(comprehensive_results)}개")
            summary.append(f"   - 성공률: {success_rate:.1f}%")
            summary.append(f"   - 평균 처리 시간: {avg_time:.2f}초")
            summary.append(f"   - 평균 신뢰도: {avg_confidence:.2f}")
            summary.append(f"   - 평균 출처 수: {sum(r.source_count for r in successful_tests) / len(successful_tests):.1f}개")
    
    # 배포 준비도
    if 'readiness_checker' in locals():
        deployment_score = readiness_checker.get_deployment_score()
        summary.append(f"\n🚀 배포 준비도: {deployment_score:.1f}/100점")
        
        if deployment_score >= 90:
            summary.append(f"   🟢 프로덕션 배포 준비 완료")
        elif deployment_score >= 70:
            summary.append(f"   🟡 일부 개선 후 배포 가능")
        elif deployment_score >= 50:
            summary.append(f"   🟠 주요 개선사항 해결 필요")
        else:
            summary.append(f"   🔴 전면적인 시스템 점검 필요")
    
    # 차별화 요소
    summary.append(f"\n🌟 차별화 요소:")
    summary.append(f"   🎯 도메인별 전문화: 각 도메인에 특화된 RAG 에이전트")
    summary.append(f"   🔄 자가교정 시스템: Corrective RAG로 답변 품질 자동 개선")
    summary.append(f"   👥 품질 보증: AI+인간 이중 검증 시스템")
    summary.append(f"   🧠 지능형 라우팅: 질문 내용 기반 최적 에이전트 선택")
    summary.append(f"   📊 실시간 모니터링: 성능 및 품질 지표 추적")
    summary.append(f"   🌐 하이브리드 검색: 내부 문서 + 실시간 웹 검색")
    
    # 향후 개선사항
    summary.append(f"\n🔧 권장 개선사항:")
    summary.append(f"   📈 성능 최적화: 병렬 처리 및 캐싱 개선")
    summary.append(f"   🎨 UI/UX 개발: Streamlit 웹 앱 고도화")
    summary.append(f"   🔒 보안 강화: API 키 관리 및 액세스 제어")
    summary.append(f"   📊 분석 고도화: 사용자 피드백 기반 학습")
    summary.append(f"   🌐 다국어 지원: 영어 문서 및 질문 처리")
    summary.append(f"   📱 모바일 최적화: 반응형 웹 인터페이스")
    
    # 결론
    summary.append(f"\n🎉 결론:")
    summary.append(f"꿀스테이 RAG 시스템은 Multi-Agent 아키텍처와 Human-in-the-Loop 품질 검증을")
    summary.append(f"통합한 차세대 기업용 지식 검색 시스템으로 성공적으로 구축되었습니다.")
    summary.append(f"도메인별 전문화, Corrective RAG, 실시간 품질 평가 등의 혁신적 기능을")
    summary.append(f"통해 기존 RAG 시스템 대비 높은 정확성과 신뢰성을 제공합니다.")
    
    summary.append(f"\n✨ 이 시스템은 기업의 내부 지식 관리와 직원 질의응답을 혁신할 수 있는")
    summary.append(f"강력한 도구로, 지속적인 학습과 개선을 통해 더욱 발전할 수 있습니다.")
    
    return "\n".join(summary)

# 최종 요약 보고서 출력
print("📄 최종 요약 보고서 생성 중...\n")

final_summary_report = generate_final_summary()
print(final_summary_report)

# 테스트 완료 메시지
print("\n" + "="*80)
print("🎊 꿀스테이 RAG 시스템 통합 테스트 완료! 🎊")
print(f"⏰ 테스트 소요 시간: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("🚀 시스템이 성공적으로 구축되고 테스트되었습니다!")
print("="*80)

## 8. 테스트 결과 저장 (선택사항)

### 완료된 전체 RAG 파이프라인 컴포넌트

✅ **01_data_processing.ipynb**
- 마크다운 문서 로딩 및 파싱
- 헤더 기반 지능형 청킹
- 메타데이터 추출 및 품질 분석

✅ **02_vector_stores.ipynb**  
- 7개 도메인별 독립 ChromaDB 구축
- bge-m3 임베딩 모델 통합
- 검색 성능 최적화 및 테스트

✅ **03_agents_development.ipynb**
- Corrective RAG 메커니즘
- 8개 전문 에이전트 (7개 도메인 + 웹검색)
- 4차원 품질 평가 시스템

✅ **04_routing_integration.ipynb**
- AI 기반 질문 분석 및 도메인 분류
- 멀티 에이전트 답변 통합
- LangGraph 워크플로우 오케스트레이션

✅ **05_hitl_evaluation.ipynb**
- ReAct 기반 6차원 평가 (60점 만점)
- 인터럽트 기반 Human-in-the-Loop
- 실시간 피드백 수집 및 분석

✅ **99_full_pipeline_test.ipynb** (현재)
- 전체 시스템 통합 테스트
- 종합 성능 분석 및 벤치마크
- 배포 준비도 점검

### 🎯 시스템 핵심 특징

1. **Multi-Agent RAG**: 도메인별 전문화된 8개 에이전트
2. **Corrective RAG**: 자가교정 메커니즘으로 품질 자동 개선
3. **Human-in-the-Loop**: AI+인간 이중 검증 품질 보증
4. **Intelligent Routing**: 질문 내용 기반 최적 에이전트 선택
5. **Real-time Monitoring**: 성능 및 품질 지표 실시간 추적
6. **Hybrid Search**: 내부 문서 + 실시간 웹 검색

### 🚀 차세대 기업용 RAG 시스템 완성!

꿀스테이 RAG 시스템은 기존 RAG의 한계를 뛰어넘어 **도메인 전문화**, **품질 보증**, **지능형 라우팅**을 통해 기업용 지식 검색의 새로운 표준을 제시합니다.